In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Transformando bancos de dados em dataframes do pandas

covid_complete = pd.read_csv('/kaggle/input/corona-virus-report/covid_19_clean_complete.csv')
covid_complete.head(10)

In [ ]:
# Informações básicas do banco de dados
covid_complete.columns

In [ ]:
covid_complete.info()

In [ ]:
#Transformando coluna "dates" para formado de data
covid_complete.Date = pd.to_datetime(covid_complete.Date)
covid_complete.info()

In [ ]:
for column in covid_complete:
    num_nan = covid_complete[column].isna().sum()
    print('{} --> {}'.format(column,num_nan))

## 1° pergunta

Como progrediu a curva dos casos nas regiões da china?

In [ ]:
# quais as regiões da china que existem no dataset
china_regions_names = covid_complete[covid_complete['Country/Region'] == 'China']['Province/State']
china_regions_names.unique()

### Quero agora as 5 regiões com o maior número de casos
1. retirar toas as regiões do banco de dados
2. agrupar novo banco de dados a partir das regiões
3. somar a quantidade de casos para ver quais são os maiores

In [ ]:
#Separando todas as regiões da china do banco de dados
china_regions_totals = covid_complete[covid_complete['Country/Region'] == 'China']

#Tomando apenas as colunas numéricas, agrupando por "Province/State" utilizando a soma e 
#Ordenando valores por numero de casos confirmados
china_regions_totals = china_regions_totals.loc[:,['Province/State','Confirmed','Deaths','Recovered','Active']]
china_regions_totals = china_regions_totals.groupby('Province/State').sum().sort_values('Confirmed', ascending = False)
china_top5 = china_regions_totals.head(5)
china_top5

### Lidando com dados invalidos

*  Como apenas a coluna de provincias contém dados invalidos, vale a pena retirar ela do dataframe.
    Mas tem um problema, existem países que tem mais de uma região, posso juntar o nome da região com o país, dessa forma posso me livrar da coluna províncias

*  Para não perder informações importantes irei fazer uma cópia do banco de dados que tenha a o nome da região junto com o do país, caso este país tenha mais de uma região no banco de dados


In [ ]:
# Criando função para adicionar província ao nome do país
def noprovince(row):
    if pd.notna(row['Province/State']):
        row['Country/Region'] = row['Country/Region'] + '_' + row['Province/State']
    return row

# Copiando o banco de dados para editar sem perder informações
covid_noprovince = covid_complete.copy()

# Colocando o nome das regiões na coluna de países
covid_noprovince = covid_noprovince.apply(noprovince, axis = 1)
covid_noprovince = covid_noprovince.drop(['Province/State'], axis = 1)
covid_noprovince.head(10)

In [ ]:
covid_noprovince.loc[covid_noprovince['Country/Region'] == 'China']

### Mesclar o banco de dados "covid_19_clean_complete" com "worldometer_data"
1. Importar banco de dados
2. Mesclar os dois a partir da coluna "Contry/Region", importando apenas a coluna de população e a de continente
3. Agrupar covid_complete por Country/Region, utilizando sum como agregate function
4. agrupar valores por continente
5. criar coluna de mortes por milhão em continentes
6. fazer ranking de mortes por milhão dos continentes


In [ ]:
# Importando o dataset "worldometer_data"
worldometer = pd.read_csv('/kaggle/input/corona-virus-report/worldometer_data.csv')

# Retirandos as colunas de interesse e mesclando com o dataset "covid_19_clean_complete"
covid_comp_deaths = covid_complete[['Country/Region','Deaths']].groupby('Country/Region').sum()
covid_world = covid_comp_deaths.merge(worldometer[['Country/Region','Population','Continent']],
                                            on = 'Country/Region')
covid_world

In [ ]:
#criando os Ranking de continentes com mais mortes por milhão

#Agrupando o banco de dados por Continente 
covid_world_deaths = covid_world.groupby('Continent', as_index = False).sum()

#Criando a coluna que conta as mortes por milhão de habitantes em cada continente
covid_world_deaths['Deaths_by_Million'] = (covid_world_deaths['Deaths']/covid_world_deaths['Population'])*10**6

#Colocando os continentes em um Ranking de maior número de morte e exibindo sua colocação
covid_world_deaths = covid_world_deaths.sort_values(by = 'Deaths_by_Million', ascending = False)
covid_world_deaths['Ranking'] =[x for x in range(1,len(covid_world_deaths)+1)]
covid_world_deaths = covid_world_deaths.set_index("Ranking")
covid_world_deaths
                             

# Exercitando data visualization

### 1° Pergunta
Nas regiões "Easter Mediterran", "Americas", "Europe", Como se corportou a curva de mortes em função do tempo? É possível observar tendências?


### 1. Construir gráfico que demonstre a evolução no tempo dos casos ativos confirmados nas regiões do mundo
* Utilizar regiões "Easter Mediterran", "Americas", "Europe"
* número de casos ativos em função do tempo
* 6 linhas, uma pra cada região do mundo, com legendes

In [ ]:
#importando as bibliotecas necessárias
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
#Agrupando os dados por região
covid_who = covid_complete.groupby(by = ['Date','WHO Region']).sum().reset_index(level = 'Date')
covid_who.head(10)

In [ ]:
#Adicionando grades de referencia
sns.set_style("darkgrid")

#Criando gráficos 
plt.plot('Date','Deaths', data = covid_who.loc[covid_who.index == "Eastern Mediterranean"], c = 'y');
plt.plot('Date','Deaths', data = covid_who.loc[covid_who.index == "Europe"], c = 'b');
plt.plot('Date','Deaths', data = covid_who.loc[covid_who.index == "Americas"], c = 'g');

#Adicionando legenda e título
plt.xlabel('Date')
plt.ylabel('Deaths')

plt.legend(["Eastern Mediterranean","Europe","Americas"]);

Neste gráfico pode-se notar que em "Eastern Mediterran" as mortes por covid foram bem baixas até a data da última atualização deste dataset. Na Europa, ocorreu um crescimento mais acentuado inicialmente, em comparação com a América, mas, a partir de maio de 2020, esta curva parece parece progredir para a estabilidade. Nas Américas, a curva começa a crescer apenas após abril de 2020, contudo seu crescimento não parece ainda progredir para a estabilidade.

### 2. Plotar o gráfico de mortes nos continentes
* Utilizar dataframe "covid_world"
* Gráfico será Bar Chart sem indicação de variância

In [ ]:
#Criando bar chart
plt.figure(figsize =(10,6))
plt.bar('Continent','Deaths', data = covid_world );

plt.title('Continet Deaths')
plt.ylabel('Deaths')

A partir do gráfico é possível notar que, até a última atualização do banco de dados, a europa e a américa do sul contém a maioria das mortes por COVID-19, enquanto na Oceania ainda não foram registradas ocorrências de morte, isso pode ocorrer por diversos fatores, como um sistem de saúde melhor equipado e o contato tardio com o virus, que permitiu que medidas sanitárias fossem tomadas de forma imadiata assim que foram registrados os primeiros casos.

### Entender a relação entre o número de habitantes e as mortes por milhão nos continentes que apresentaram maior média
* Utilizar somente o dataframe "worldometer"
* Descobrir quais os 3 continentes com mais mortes por milhão de habitantes
* Produzir um dataframe que contenha apenas eles
* Fazer um gráfico Scatterplot para representar essa relação

In [ ]:
#Agrupando por continentes e descobrindo quais os 3 com a maior média de mortes por milhão de habitantes
world_most_deaths = worldometer[['Continent','Deaths/1M pop']].groupby('Continent', as_index = False).mean().sort_values('Deaths/1M pop',ascending = False)
world_most_deaths

### Então iremos utilizar apenas os dados da América do Sul, Europa e Ásia
É importante observar que não precisamos de todo o banco de dados então só retiraremos as colunas necessárias
1. Continent
2. Population
3. Deaths/1M pop

In [ ]:
#Retirando os dados que queremos do banco de dados geral
pop_x_deaths = worldometer.loc[(worldometer['Continent'] == 'South America')|
                               (worldometer['Continent'] =='Europe')|
                               (worldometer['Continent'] =='North America'),
                              ['Continent','Population','Deaths/1M pop']]
pop_x_deaths

In [ ]:
#Criando gráfico de interações
plt.figure(figsize =(12,6))
plt.title('Relação entre Mosrtes por milhão e população total do país')

sns.scatterplot(x = 'Population',
               y = 'Deaths/1M pop',
               hue = 'Continent',
               data = pop_x_deaths);

Primeiramente, é possível notar que na maioria dos países nestes continentes, a população é menor que 50 milhões de habitantes, nestes casos, existem países na europa que, mesmo com um população relativamente baixa, observaram um alto número de mortes por milhão, enquanto na américa do norte e do sul a população a taxa de mortes por milhão não demonstrou um número tão elevado. Uma das hipóteses para isso é que os países europeus contém uma alta concentração de pessoas em áreas menores, se comprada às américas, o que pode facilitar a propagação do vírus. 

### Criando Heatmap de correlação
1. Utilizado o método ```DataFrame.corr()``` do pandas para identificar a correlação entre features
2. Utilizar o heatmap do seaborn para visualização das correlações 

In [ ]:

fig, ax = plt.subplots(figsize=(15, 15))
plt.title('Correlação entre colunas')
sns.heatmap(worldometer.corr(), annot=True, square=True, cmap='coolwarm');

1. O valor da população tem uma alta correlação com novos casos, novas mosrtes e novas recuperações, mas não tanto com valores totais. Este fato pode mostrar que uma maior população interfere diretamente na taxa com que o virus cresce, não necessáriamente nos valores totais, considerando as informações até a última atualização do banco de dados
2. Os Testes por milhão de habitantes tem uma alta correlação negativa com os novos casos, novas mortes e novas recuperações. Uma das hipóteses possíveis é que esta característica do banco de dados pode agir de forma contrária à população, diminuindo a taxa com que o virus se propaga, já que, com uma maior taxa de teste em relação a população, pode-se ter medidas sanitárias masi acertivas e localizadas para a contenção do vírus. 